In [1]:
# ! cat /proc/cpuinfo

## 라이브러리 로딩

In [1]:

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 다운로드

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import datetime

# 데이터 로드

In [4]:
path = "/content/drive/MyDrive/Colab Notebooks/cw_dacon/gas/"

In [5]:
total = pd.read_csv(path+'20181231.csv', encoding='cp949')

In [6]:
total.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105


In [6]:
d_map = {}
for i, d in enumerate(total['구분'].unique()):
    d_map[d] = i
total['구분'] = total['구분'].map(d_map)

In [7]:
total['연월일'] = pd.to_datetime(total['연월일'])

In [8]:
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

In [9]:
def plus_time(x):
    return x['연월일'] + datetime.timedelta(hours=x['시간'])
total['time'] = total.apply(plus_time, axis=1)
total

,연월일,시간,구분,공급량,year,month,day,weekday,time
0,2013-01-01,1,0,2497.129,2013,1,1,1,2013-01-01 01:00:00
1,2013-01-01,2,0,2363.265,2013,1,1,1,2013-01-01 02:00:00
2,2013-01-01,3,0,2258.505,2013,1,1,1,2013-01-01 03:00:00
3,2013-01-01,4,0,2243.969,2013,1,1,1,2013-01-01 04:00:00
4,2013-01-01,5,0,2344.105,2013,1,1,1,2013-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,2018-12-31 20:00:00
368084,2018-12-31,21,6,669.961,2018,12,31,0,2018-12-31 21:00:00
368085,2018-12-31,22,6,657.941,2018,12,31,0,2018-12-31 22:00:00
368086,2018-12-31,23,6,610.953,2018,12,31,0,2018-12-31 23:00:00


In [10]:
total_a = total[total['구분']==0].loc[:,['time','공급량']].reset_index(drop=True,inplace=False)
total_b = total[total['구분']==1].loc[:,['time','공급량']].reset_index(drop=True,inplace=False)
# total_c = total[total['구분']==2].loc[:,['time','공급량']].reset_index(drop=True,inplace=False)
# total_d = total[total['구분']==3].loc[:,['time','공급량']].reset_index(drop=True,inplace=False)
# total_e = total[total['구분']==4].loc[:,['time','공급량']].reset_index(drop=True,inplace=False)
# total_g = total[total['구분']==5].loc[:,['time','공급량']].reset_index(drop=True,inplace=False)
# total_h = total[total['구분']==6].loc[:,['time','공급량']].reset_index(drop=True,inplace=False)

In [11]:
total_a.columns = ['time','공급량_a']
total_b.columns = ['time','공급량_b']
# total_c.columns = ['time','공급량_c']
# total_d.columns = ['time','공급량_d']
# total_e.columns = ['time','공급량_e']
# total_g.columns = ['time','공급량_g']
# total_h.columns = ['time','공급량_h']
# total_h

In [12]:
train = pd.concat([total_a,total_b.loc[:,['공급량_b']]],axis=1)
# train = pd.concat([train,total_c.loc[:,['공급량_c']]],axis=1)
# train = pd.concat([train,total_d.loc[:,['공급량_d']]],axis=1)
# train = pd.concat([train,total_e.loc[:,['공급량_e']]],axis=1)
# train = pd.concat([train,total_g.loc[:,['공급량_g']]],axis=1)
# train = pd.concat([train,total_h.loc[:,['공급량_h']]],axis=1)
# train

### 예측해야될 데이터
---
- dangjin_floating : 당진수상태양광 발전량(KW)
- dangjin_warehouse : 당진자재창고태양광 발전량(KW)
- dangjin : 당진태양광 발전량(KW)
- ulsan : 울산태양광 발전량(KW)

In [ ]:
#dangjin_fcst_data = pd.read_csv('data/dangjin_fcst_data.csv') #당진지역 발전소 동네 예보
#dangjin_obs_data = pd.read_csv('data/dangjin_obs_data.csv') # 당진지역 발전소 인근 기상 관측 자료

#ulsan_fcst_data = pd.read_csv('data/ulsan_fcst_data.csv') # 울산지역 발전소 동네 에보
#ulsan_obs_data = pd.read_csv('data/ulsan_obs_data.csv') # 울산지역 인근 기상 관측 자료

#site_info = pd.read_csv('data/site_info.csv') #발전소 정보

# energy = pd.read_csv('data/energy.csv') # 발전소별 발전량

### 결측치 처리

In [ ]:
#dangjin_obs_data.fillna(dangjin_obs_data.mean(),inplace = True)
# ulsan_obs_data.fillna(ulsan_obs_data.mean(),inplace = True)
energy.fillna(energy.mean(),inplace = True)

### energy의 time를 index로 지정

In [ ]:
# energy.head()

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2018-03-01 1:00:00,0.0,0.0,0,0
1,2018-03-01 2:00:00,0.0,0.0,0,0
2,2018-03-01 3:00:00,0.0,0.0,0,0
3,2018-03-01 4:00:00,0.0,0.0,0,0
4,2018-03-01 5:00:00,0.0,0.0,0,0


In [13]:
train = train.set_index('time')
train.head()

,공급량_a,공급량_b
time,,
2013-01-01 01:00:00,2497.129,2169.093
2013-01-01 02:00:00,2363.265,2051.733
2013-01-01 03:00:00,2258.505,1923.253
2013-01-01 04:00:00,2243.969,1927.253
2013-01-01 05:00:00,2344.105,2047.333


In [45]:
train

,공급량_a,공급량_b
time,,
2013-01-01 01:00:00,2497.129,2169.093
2013-01-01 02:00:00,2363.265,2051.733
2013-01-01 03:00:00,2258.505,1923.253
2013-01-01 04:00:00,2243.969,1927.253
2013-01-01 05:00:00,2344.105,2047.333
...,...,...
2018-12-31 20:00:00,3040.868,2668.688
2018-12-31 21:00:00,2973.577,2606.681
2018-12-31 22:00:00,2840.272,2540.169


In [ ]:
# train[train['공급량_a']==2345.481]

,공급량_a,공급량_b,공급량_c,공급량_d,공급량_e,공급량_g,공급량_h
time,,,,,,,
2013-01-02 01:00:00,2345.481,1991.733,221.378,1291.316,3140.389,3205.085,521.756


In [ ]:
# train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52584 entries, 2013-01-01 01:00:00 to 2019-01-01 00:00:00
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   공급량_a   52584 non-null  float64
 1   공급량_b   52584 non-null  float64
 2   공급량_c   52584 non-null  float64
 3   공급량_d   52584 non-null  float64
 4   공급량_e   52584 non-null  float64
 5   공급량_g   52584 non-null  float64
 6   공급량_h   52584 non-null  float64
dtypes: float64(7)
memory usage: 3.2 MB


## 모델링

In [32]:
# 데이터 타입을 float로 변경

train_a = train['공급량_a'].values.astype(float)
# train_b = train['공급량_b'].values.astype(float)
# train_c = train['공급량_c'].values.astype(float)
# train_d = train['공급량_d'].values.astype(float)
# train_e = train['공급량_e'].values.astype(float)
# train_g = train['공급량_g'].values.astype(float)
# train_h = train['공급량_h'].values.astype(float)



'''valid_data_size = 700
train_data = ulsan[:-valid_data_size]
valid_data = ulsan[-valid_data_size:]'''

'valid_data_size = 700\ntrain_data = ulsan[:-valid_data_size]\nvalid_data = ulsan[-valid_data_size:]'

### 하이퍼 파라미터 설정

In [15]:
# hyper parameters

learning_rate = 0.0001 
sequence_length = 24*365 # 24 일때가 가장 좋았음.
epochs = 10

In [16]:
# 데이터를 시퀀스 형태로 변경
def make_batch(input_data, sl):
    train_x = []
    train_y = []
    L = len(input_data)
    for i in range(L-sl):
        train_seq = input_data[i:i+sl]
        train_label = input_data[i+sl:i+sl+1]
        train_x.append(train_seq)
        train_y.append(train_label)
    return train_x, train_y

In [27]:
## 모델 설계

class simple_lstm(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.input_vector = 1
        self.sequence_length = 24
        self.output_vector = 100
        self.num_layers = 4
        
        self.lstm = nn.LSTM(input_size=self.input_vector, hidden_size=self.output_vector, num_layers=self.num_layers, batch_first=True)
        self.linear = nn.Sequential(
            nn.Linear(self.output_vector, 1)
        )
        # self.linear = nn.Sequential(
        #     nn.Linear(self.output_vector, 50),
        #     nn.Linear(50, 30),
        #     nn.Linear(30, 10),
        #     nn.Linear(10,1)
        # )
        
    def forward(self, x):
        output, _ = self.lstm(x) #(hidden, cell) 데이터는 사용하지 않음
        return self.linear(output[:,-1,:])
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### total_a

In [33]:
train_x, train_y = make_batch(train_a.reshape(-1,1), sequence_length)
tensor_x = torch.Tensor(train_x).to(torch.int64)
tensor_y = torch.Tensor(train_y).to(torch.int64)

In [37]:
tensor_x

tensor([[[2497],
         [2363],
         [2258],
         ...,
         [2097],
         [1999],
         [1858]],

        [[2363],
         [2258],
         [2243],
         ...,
         [1999],
         [1858],
         [1677]],

        [[2258],
         [2243],
         [2344],
         ...,
         [1858],
         [1677],
         [1567]],

        ...,

        [[2054],
         [1981],
         [1897],
         ...,
         [2991],
         [3040],
         [2973]],

        [[1981],
         [1897],
         [1765],
         ...,
         [3040],
         [2973],
         [2840]],

        [[1897],
         [1765],
         [1679],
         ...,
         [2973],
         [2840],
         [2692]]], device='cuda:0')

In [34]:
train_a = simple_lstm()
train_a = train_a.to(device)

In [35]:
optimizer = torch.optim.Adam(train_a.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [21]:
!nvidia-smi    #GPU 사용량

Mon Dec  6 05:50:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    32W / 250W |   1035MiB / 16280MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [38]:
import torch, gc

for i in range(epochs):
    # gc.collect()
    # torch.cuda.empty_cache()
    train_a.train()
    tensor_x = tensor_x.to(device)
    tensor_y = tensor_y.to(device)
    output = train_a(tensor_x)
    loss = criterion(output, tensor_y.view(-1,1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))

RuntimeError: ignored

In [ ]:
x_input = np.array(train.train_a[-24:]) #  next value based on data of last year
x_input = x_input.reshape((1, 24, 1)) 
train_a_pred = []

In [ ]:
for i in range(2160):   #2160  #672=24*28
    
    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = train_a(x_input).cpu().detach().numpy()
    
    new_input = predict.reshape((1,1,1))
    x_input = np.concatenate((x_input[:,-23:].cpu(), new_input), axis = 1)
    train_a_pred.append(predict[0][0])

### dangjin_warehouse

In [ ]:
train_x, train_y = make_batch(dangjin_warehouse.reshape(-1,1), sequence_length)
tensor_x = torch.Tensor(train_x)
tensor_y = torch.Tensor(train_y)


dangjin_warehouse = simple_lstm()
dangjin_warehouse = dangjin_warehouse.to(device)

optimizer = torch.optim.Adam(dangjin_warehouse.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

for i in range(epochs):
    dangjin_warehouse.train()
    tensor_x = tensor_x.to(device)
    tensor_y = tensor_y.to(device)
    output = dangjin_warehouse(tensor_x)
    loss = criterion(output, tensor_y.view(-1,1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))

x_input = np.array(energy.dangjin_warehouse[-24:]) #  next value based on data of last year
x_input = x_input.reshape((1, 24, 1)) 
dangjin_warehouse_pred= []

for i in range(672):
    
    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = dangjin_warehouse(x_input).cpu().detach().numpy()
    
    new_input = predict.reshape((1,1,1))
    x_input = np.concatenate((x_input[:,-23:].cpu(), new_input), axis = 1)
    dangjin_warehouse_pred.append(predict[0][0])

Epoch 0, Loss 29991.69531
Epoch 100, Loss 29237.04297
Epoch 200, Loss 25498.15234
Epoch 300, Loss 21772.12695
Epoch 400, Loss 21364.16602
Epoch 500, Loss 9767.13770
Epoch 600, Loss 5690.01953
Epoch 700, Loss 2897.01001
Epoch 800, Loss 1666.82739
Epoch 900, Loss 1174.23206
Epoch 1000, Loss 1031.15967
Epoch 1100, Loss 988.46277
Epoch 1200, Loss 968.51605
Epoch 1300, Loss 955.71423
Epoch 1400, Loss 945.90784
Epoch 1500, Loss 939.43378
Epoch 1600, Loss 930.69183
Epoch 1700, Loss 926.46777
Epoch 1800, Loss 919.19995
Epoch 1900, Loss 914.06439


### dangjin

In [ ]:
train_x, train_y = make_batch(dangjin.reshape(-1,1), sequence_length)
tensor_x = torch.Tensor(train_x)
tensor_y = torch.Tensor(train_y)


dangjin = simple_lstm()
dangjin = dangjin.to(device)

optimizer = torch.optim.Adam(dangjin.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

for i in range(epochs):
    dangjin.train()
    tensor_x = tensor_x.to(device)
    tensor_y = tensor_y.to(device)
    output = dangjin(tensor_x)
    loss = criterion(output, tensor_y.view(-1,1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))

x_input = np.array(energy.dangjin[-24:]) #  next value based on data of last year
x_input = x_input.reshape((1, 24, 1)) 
dangjin_pred = []

for i in range(672):   
    
    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = dangjin(x_input).cpu().detach().numpy()
    new_input = predict.reshape((1,1,1))
    x_input = np.concatenate((x_input[:,-23:].cpu(), new_input), axis = 1)
    dangjin_pred.append(predict[0][0])

Epoch 0, Loss 68170.28125
Epoch 100, Loss 67084.38281
Epoch 200, Loss 60498.91406
Epoch 300, Loss 51220.14453
Epoch 400, Loss 48626.66406
Epoch 500, Loss 48547.03125
Epoch 600, Loss 21144.20703
Epoch 700, Loss 6794.92529
Epoch 800, Loss 3000.24561
Epoch 900, Loss 2467.31519
Epoch 1000, Loss 2312.99414
Epoch 1100, Loss 2228.74951
Epoch 1200, Loss 2179.66504
Epoch 1300, Loss 2145.10010
Epoch 1400, Loss 2119.58081
Epoch 1500, Loss 2084.83594
Epoch 1600, Loss 2062.43530
Epoch 1700, Loss 2042.13440
Epoch 1800, Loss 2014.49255
Epoch 1900, Loss 1989.32190


### ulsan

In [ ]:
train_x, train_y = make_batch(ulsan.reshape(-1,1), sequence_length)
tensor_x = torch.Tensor(train_x)
tensor_y = torch.Tensor(train_y)


ulsan = simple_lstm()
ulsan = ulsan.to(device)

optimizer = torch.optim.Adam(ulsan.parameters(), lr=learning_rate)
criterion = nn.MSELoss()


# 학습
for i in range(epochs):
    ulsan.train()
    tensor_x = tensor_x.to(device)
    tensor_y = tensor_y.to(device)
    output = ulsan(tensor_x)
    loss = criterion(output, tensor_y.view(-1,1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))

        
        
# 추론        
x_input = np.array(energy.ulsan[-24:]) #  next value based on data of last year
x_input = x_input.reshape((1, 24, 1)) 
ulsan_pred = []

for i in range(672):
    
    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = ulsan(x_input).cpu().detach().numpy()
    new_input = predict.reshape((1,1,1))
    x_input = np.concatenate((x_input[:,-23:].cpu(), new_input), axis = 1)
    ulsan_pred.append(predict[0][0])

Epoch 0, Loss 14324.69922
Epoch 100, Loss 13653.27637
Epoch 200, Loss 11321.62500
Epoch 300, Loss 10213.53027
Epoch 400, Loss 10195.78027
Epoch 500, Loss 6182.88281
Epoch 600, Loss 1220.37268
Epoch 700, Loss 565.07166
Epoch 800, Loss 479.22906
Epoch 900, Loss 453.16119
Epoch 1000, Loss 441.70035
Epoch 1100, Loss 435.23682
Epoch 1200, Loss 430.89359
Epoch 1300, Loss 427.13724
Epoch 1400, Loss 424.04031
Epoch 1500, Loss 425.34241
Epoch 1600, Loss 419.32108
Epoch 1700, Loss 416.37091
Epoch 1800, Loss 413.65601
Epoch 1900, Loss 411.39694


In [ ]:
submission = pd.read_csv('data/sample_submission.csv')

In [ ]:
submission.iloc[:24*28, 1] = train_a_pred
submission.iloc[:24*28, 2] = dangjin_warehouse_pred
submission.iloc[:24*28, 3] = dangjin_pred
submission.iloc[:24*28, 4] = ulsan_pred

In [ ]:
submission.to_csv('submission.csv',index=False)

In [ ]:
submission

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0.772095,-0.162957,-1.030323,0.225382
1,2021-02-01 02:00:00,1.358530,-0.859719,-3.343087,0.394613
2,2021-02-01 03:00:00,1.163334,-2.927019,-7.431317,0.633824
3,2021-02-01 04:00:00,-0.558547,-6.900148,-11.184819,1.306655
4,2021-02-01 05:00:00,-5.397500,-10.482171,-12.712384,3.901035
...,...,...,...,...,...
1387,2021-07-08 20:00:00,0.000000,0.000000,0.000000,0.000000
1388,2021-07-08 21:00:00,0.000000,0.000000,0.000000,0.000000
1389,2021-07-08 22:00:00,0.000000,0.000000,0.000000,0.000000
1390,2021-07-08 23:00:00,0.000000,0.000000,0.000000,0.000000


## reference

[간단한 CNN 모델](https://dacon.io/competitions/official/235720/codeshare/2563?page=1&dtype=recent)    
[RNN time-series 예측](http://cedartrees.co.kr/index.php/2020/08/19/rnn-time-series-prediction1/)  